## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Thompson,CA,55.7435,-97.8558,50.16,62,40,10.36,scattered clouds
1,1,Pochutla,MX,15.7432,-96.4661,85.82,65,91,5.48,overcast clouds
2,2,Saint-Philippe,RE,-21.3585,55.7679,67.55,80,98,13.38,light rain
3,3,Valle Hermoso,MX,25.6667,-97.8333,95.14,57,2,17.54,clear sky
4,4,Mataura,NZ,-46.1927,168.8643,49.62,87,99,3.80,moderate rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
18,18,Riyadh,SA,24.6877,46.7219,78.94,17,0,8.23,clear sky
22,22,Butaritari,KI,3.0707,172.7902,82.98,71,86,15.03,overcast clouds
24,24,Salinopolis,BR,-0.6136,-47.3561,81.09,75,77,11.52,broken clouds
31,31,El Sauce,NI,12.8869,-86.5390,82.60,80,78,3.11,broken clouds
33,33,Bengkulu,ID,-3.8004,102.2655,76.03,88,96,2.35,overcast clouds
38,38,Faanui,PF,-16.4833,-151.7500,78.40,73,25,12.01,scattered clouds
40,40,Lorengau,PG,-2.0226,147.2712,80.78,79,100,1.70,overcast clouds
43,43,Dubuque,US,42.4833,-90.8668,82.22,61,20,21.85,few clouds
47,47,Posadas,AR,-27.3671,-55.8961,80.58,78,0,14.97,clear sky
54,54,Bontang,ID,0.1333,117.5000,76.48,87,85,3.71,moderate rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                137
City                   137
Country                137
Lat                    137
Lng                    137
Max Temp               137
Humidity               137
Cloudiness             137
Wind Speed             137
Current Description    137
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
# hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
18,Riyadh,SA,78.94,clear sky,24.6877,46.7219,
22,Butaritari,KI,82.98,overcast clouds,3.0707,172.7902,
24,Salinopolis,BR,81.09,broken clouds,-0.6136,-47.3561,
31,El Sauce,NI,82.60,broken clouds,12.8869,-86.5390,
33,Bengkulu,ID,76.03,overcast clouds,-3.8004,102.2655,
38,Faanui,PF,78.40,scattered clouds,-16.4833,-151.7500,
40,Lorengau,PG,80.78,overcast clouds,-2.0226,147.2712,
43,Dubuque,US,82.22,few clouds,42.4833,-90.8668,
47,Posadas,AR,80.58,clear sky,-27.3671,-55.8961,
54,Bontang,ID,76.48,moderate rain,0.1333,117.5000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# Explore the hotel_df Data Frame
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
18,Riyadh,SA,78.94,clear sky,24.6877,46.7219,"InterContinental Riyadh, an IHG Hotel"
22,Butaritari,KI,82.98,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
24,Salinopolis,BR,81.09,broken clouds,-0.6136,-47.3561,Salinas Park Resort Maçarico AP101
31,El Sauce,NI,82.60,broken clouds,12.8869,-86.5390,Hotel Blanco
33,Bengkulu,ID,76.03,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
38,Faanui,PF,78.40,scattered clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
40,Lorengau,PG,80.78,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
43,Dubuque,US,82.22,few clouds,42.4833,-90.8668,American Inn and Suites
47,Posadas,AR,80.58,clear sky,-27.3671,-55.8961,Hotel Posadas
54,Bontang,ID,76.48,moderate rain,0.1333,117.5000,KOPKAR Hotel Bintang Sintuk


In [10]:
# Explore the hotel_df Data Frame for empty rows
hotel_df.count()

City                   137
Country                137
Max Temp               137
Current Description    137
Lat                    137
Lng                    137
Hotel Name             137
dtype: int64

In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Define marker layer using gmaps documentation and adding info content box
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add markers layer
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))